In [ ]:
def derive_coupling_map_from_circuit(transpiled_circuit):
    derived_coupling_map = set()  # Use a set to avoid duplicate entries

    # Initialize the swap history for each qubit in the circuit
    qubit_swap_history = {qubit: False for qubit in transpiled_circuit.qubits}

    for instruction in transpiled_circuit.data:
        if instruction[0].name == 'swap':
            # If the instruction is a SWAP gate, mark the qubits as involved in a swap
            for qarg in instruction[1]:
                qubit_swap_history[qarg] = True
            continue  # Move to the next instruction

        if len(instruction[1]) == 2:
            # Directly use the qubit objects
            qubits = [qarg for qarg in instruction[1]]

            # Check if any of the qubits were involved in a SWAP
            if not any(qubit_swap_history[qubit] for qubit in qubits):
                # If neither qubit has been swapped, add the connection
                qubit_indices = [transpiled_circuit.find_bit(qarg).index for qarg in qubits]
                derived_coupling_map.add(tuple(sorted(qubit_indices)))

    return derived_coupling_map

In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit.library import CXGate
# from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler import CouplingMap
from qiskit.circuit.library import C4XGate
from qiskit_aer import AerSimulator
# from qiskit.providers.fake_provider import FakeCambridge
QiskitRuntimeService.delete_account()

import time
import os
# QiskitRuntimeService.save_account(token="IYxMCq8Fhs5NzG80K3br2EhvbU5WziWlfypyuyr3SELB",channel="ibm_quantum",overwrite=True )
# Set the API key as an environment variable

# Initialize the Qiskit Runtime Service
your_api_key = "V_5F6RZPUb9GB37hSH3CQrQpdFX7WpfP2kek91zjo0Ml"
your_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/5e194ca79679454da57a2ad2d3e49a9d:749f65fa-d77a-44f6-913a-5a77858ae7aa::"


QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token=your_api_key,
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/5e194ca79679454da57a2ad2d3e49a9d:749f65fa-d77a-44f6-913a-5a77858ae7aa::",
    overwrite=True
)
service = QiskitRuntimeService()
# Specify the backend
backend_name = 'ibm_brisbane'
backend = service.backend(backend_name)

sim =AerSimulator()
from collections import Counter, defaultdict
from qiskit.circuit.random import random_circuit
from qiskit import transpile
import numpy as np
from qiskit.circuit.random import random_circuit
random_circ = random_circuit(num_qubits=100, depth=5, max_operands=2)
random_circ= transpile(random_circ, backend=backend, seed_transpiler=42)

big_rc= derive_coupling_map_from_circuit(random_circ)
print(big_rc)


random_circ = random_circuit(num_qubits=100, depth=5, max_operands=2)
random_circ= transpile(random_circ, backend=backend, seed_transpiler=40)

big_rc_2= derive_coupling_map_from_circuit(random_circ)

random_circ = random_circuit(num_qubits=100, depth=5, max_operands=2)
random_circ= transpile(random_circ, backend=backend, seed_transpiler=43)

big_rc_3= derive_coupling_map_from_circuit(random_circ)

random_circ = random_circuit(num_qubits=100, depth=5, max_operands=2)
random_circ= transpile(random_circ, backend=backend, seed_transpiler=52)

big_rc_4= derive_coupling_map_from_circuit(random_circ)

random_circ = random_circuit(num_qubits=100, depth=5, max_operands=2)
random_circ= transpile(random_circ, backend=backend, seed_transpiler=49)

big_rc_5= derive_coupling_map_from_circuit(random_circ)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
coupling_maps=[]
# Define the derived coupling maps as lists of tuples (edges)
# Assuming derived_map_grover_l, derived_map_t_qft, derived_map_t_1bit, derived_map_1bit_adder are defined
coupling_maps.append(big_rc_5)
coupling_maps.append(big_rc_4)
coupling_maps.append(big_rc_2)
coupling_maps.append(big_rc_3)
coupling_maps.append(big_rc)


from collections import Counter
import pandas as pd

# Flatten the list of edges from all coupling maps
all_edges = [edge for cmap in coupling_maps for edge in cmap]

# Count occurrences of each edge
edge_counts = Counter(all_edges)

# Sort edges by count in descending order
sorted_edges = sorted(edge_counts.items(), key=lambda x: x[1], reverse=True)

# Assign ranks correctly
ranked_edges = []
current_rank = 0  # The displayed rank
prev_count = None  # Track previous count

for edge, count in sorted_edges:
    if prev_count is None or count < prev_count:
        current_rank = current_rank + 1  # Ensure consecutive ranking
    
    ranked_edges.append(("{}_{}".format(edge[0], edge[1]), count, current_rank))
    prev_count = count  # Update previous count

# Create a DataFrame for structured output
df = pd.DataFrame(ranked_edges, columns=["Qubit Pair", "Count", "Rank"])

# Print formatted table
print(df.to_string(index=False))

    
combined_coupling_map = big_rc|big_rc_4|big_rc_2|big_rc_3|big_rc_5

In [ ]:
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\rupsh\Downloads\ibm_brisbane_calibrations_2025-05-05T15_33_39Z.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Clean column names by stripping spaces
df.columns = df.columns.str.strip()

# Extract and rank ECR edges based on error rates
edge_errors = []

# Iterate through the "ECR error" column
for ecr_data in df["ECR error"].dropna():
    pairs = ecr_data.split(";")  # Split multiple entries
    for pair in pairs:
        qubits, error = pair.split(":")
        edge_errors.append((qubits, float(error)))

# Convert to DataFrame and sort by error rate
edge_errors_df = pd.DataFrame(edge_errors, columns=["Qubit Pair", "ECR Error"])
edge_errors_df = edge_errors_df.sort_values(by="ECR Error", ascending=True)
edge_errors_df["Rank"] = range(1, len(edge_errors_df) + 1)

pd.set_option("display.max_rows", None)  # Show all rows

# Display the ranked edges
print(edge_errors_df)



In [ ]:
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\rupsh\Downloads\ibm_brisbane_calibrations_2025-05-05T15_33_39Z.csv"  # Replace with your actual file path
df_errors = pd.read_csv(file_path)

# Clean column names by stripping spaces
df_errors.columns = df_errors.columns.str.strip()
# Extract and rank ECR edges based on error rates
edge_errors = []

for ecr_data in df_errors["ECR error"].dropna():
    pairs = ecr_data.split(";")  # Split multiple entries
    for pair in pairs:
        qubits, error = pair.split(":")
        edge_errors.append((qubits.strip(), float(error)))

# Convert to DataFrame and sort by error rate
df_errors = pd.DataFrame(edge_errors, columns=["Qubit Pair", "ECR Error"])
df_errors = df_errors.sort_values(by="ECR Error", ascending=True).reset_index(drop=True)

# Assign group ranks: group of 10 edges per rank group
group_size = 24
df_errors["Group_Rank_ECR"] = (df_errors.index // group_size) + 1
# df_errors = df_errors.sort_values(by="ECR Error", ascending=True)
# df_errors["Rank_ECR"] = range(1, len(df_errors) + 1)
df_counts = df[["Qubit Pair", "Rank"]].rename(columns={"Rank": "Rank_Counts"}).copy()
df_counts = df_counts.sort_values(by="Rank_Counts").reset_index(drop=True)
df_counts["Group_Rank_Counts"] = (df_counts.index // group_size) + 1

# Merge on "Qubit Pair"
df_comparison = pd.merge(
    df_counts[["Qubit Pair", "Group_Rank_Counts"]],
    df_errors[["Qubit Pair", "Group_Rank_ECR"]],
    on="Qubit Pair",
    how="inner"
)

# Compute the absolute group rank difference
df_comparison["Rank Difference"] = abs(df_comparison["Group_Rank_Counts"] - df_comparison["Group_Rank_ECR"])

# Sort and plot
df_comparison_sorted = df_comparison.sort_values(by="Rank Difference", ascending=True)
max_diff = df_comparison["Rank Difference"].max()
bins = np.arange(0, max_diff + 2) - 0.5  # shift bins so ticks are centered under bars
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5, 3))

# Plot distribution with KDE
sns.histplot(df_comparison["Rank Difference"], kde=True, bins=20, color="grey")

plt.xlabel("Rank Difference")
plt.ylabel("Density")
plt.title("Bin Rank Differences (Brisbane)")

plt.show()



